In [1]:
from lib import *
import numpy as np

import matplotlib.pyplot as plt
import plotly.figure_factory as ff
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from itertools import product

*** THIS NOTEBOOK CAN BE USED TO ANALYSE RAW AND DEC OP.WVFs FROM LARSOFT ***

In [2]:
SER = []
templates = ["fbk_digi.txt","fbk_1024_dapv2.txt","fbk_DAPHNE_Dec22.txt","fbk_CA_deco_sinpret.txt"]
for path in templates:
    SER.append(np.loadtxt("template/"+path))

In [13]:
fig = go.Figure()
for i in range(len(SER)):
    fig.add_trace(go.Scatter(name=templates[i],x=np.arange(len(SER[i])),y=SER[i]))
fig.show()

In [4]:
raw,key_list         = load_larsoft_root("RAW","opdetraw_hist.root",     "opdigiana")
gauss_dec,key_list   = load_larsoft_root("DEC","deconv_gauss_hist.root", "opdecoana")
wiener_dec,key_list  = load_larsoft_root("DEC","deconv_wiener_hist.root","opdecoana")

/home/smanthey/.local/lib/python3.10/site-packages/numpy/core/fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/smanthey/.local/lib/python3.10/site-packages/numpy/core/_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)
/home/smanthey/.local/lib/python3.10/site-packages/numpy/core/_methods.py:269: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/smanthey/.local/lib/python3.10/site-packages/numpy/core/_methods.py:226: RuntimeWarning: invalid value encountered in divide
  arrmean = um.true_divide(arrmean, div, out=arrmean,
/home/smanthey/.local/lib/python3.10/site-packages/numpy/core/_methods.py:261: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [5]:
ordered_raw         = order_wvfs(gauss_dec,key_list)
ordered_gauss_dec   = order_wvfs(gauss_dec,key_list)
ordered_wiener_dec  = order_wvfs(gauss_dec,key_list)

*** VISUALIZE DATA ***

In [6]:
num = np.random.randint(len(raw["RECO"]["WVF"]))
# print(num)

fig = make_subplots(rows=1, cols=2,
                    # subplot_titles=('Digi wvf', 'Deco wvf')
                    )

fig.add_trace(go.Line(name="RAW #PE: %i"%(raw["RECO"]["PE"][num]),x=raw["RECO"]["WVF_X"][num],y=raw["RECO"]["WVF"][num]-raw["PEDESTAL"]),col=1,row=1)
fig.add_trace(go.Histogram(name="TRUE PHOTONS",x=raw["RECO"]["PETIMES"][num]),col=1,row=1)
fig.add_trace(go.Line(name="GAUSS #PE: %.2f"%(gauss_dec["RECO"]["PE"][num]),x=gauss_dec["RECO"]["WVF_X"][num],y=gauss_dec["RECO"]["WVF"][num]/1e3),col=2,row=1)
fig.add_trace(go.Line(name="WIENER #PE: %.2f"%(wiener_dec["RECO"]["PE"][num]),x=wiener_dec["RECO"]["WVF_X"][num],y=wiener_dec["RECO"]["WVF"][num]),col=2,row=1)
fig.add_vline(x=gauss_dec["RECO"]["PETIMES"][num]*1e6, line_width=2, line_dash="dash", line_color="gray",col=2)

fig.update_layout(
    title = "Deco. wvf comparison",
    xaxis_title  = "Time in [&mu;s]",
    yaxis_title  = "Amp. in [ADC]",
    xaxis2_title = "Time in [&mu;s]",
    yaxis2_title = "Amp. in [a.u.]",
)
fig.show()

/home/smanthey/.local/lib/python3.10/site-packages/plotly/graph_objs/_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




*** CONFIGURE FILTER FOR ANALYSIS ***

In [7]:
# PE_filter = np.asarray(raw["RECO"]["PE"]) != 0
PE_filter = np.asarray(raw["RECO"]["PE"]) > 100

In [8]:
PE_error_gauss = (np.asarray(gauss_dec["RECO"]["PE"])[PE_filter]-np.asarray(raw["RECO"]["PE"])[PE_filter])/np.asarray(raw["RECO"]["PE"])[PE_filter]
PE_error_wiener = (np.asarray(wiener_dec["RECO"]["PE"])[PE_filter]-np.asarray(raw["RECO"]["PE"])[PE_filter])/np.asarray(raw["RECO"]["PE"])[PE_filter]

fig = make_subplots(rows=1, cols=3,
# subplot_titles=('Amp study', 'Charge study')
)

fig.add_trace(go.Histogram(name="GAUSS AMP", x=np.asarray(raw["RECO"]["AMP"]       )[PE_filter]),row=1,col=1)
fig.add_trace(go.Histogram(name="GAUSS AMP", x=np.asarray(gauss_dec["RECO"]["AMP"] )[PE_filter]),row=1,col=1)
fig.add_trace(go.Histogram(name="WIENER AMP",x=np.asarray(wiener_dec["RECO"]["AMP"])[PE_filter]),row=1,col=1)

fig.add_trace(go.Scatter(name="RAW PE vs AMP",   mode="markers",x=np.asarray(raw["RECO"]["AMP"]       )[PE_filter],y=np.asarray(raw["RECO"]["PE"]       )[PE_filter]),row=1,col=2)
fig.add_trace(go.Scatter(name="GAUSS PE vs AMP", mode="markers",x=np.asarray(gauss_dec["RECO"]["AMP"] )[PE_filter],y=np.asarray(gauss_dec["RECO"]["PE"] )[PE_filter]),row=1,col=2)
fig.add_trace(go.Scatter(name="WIENER PE vs AMP",mode="markers",x=np.asarray(wiener_dec["RECO"]["AMP"])[PE_filter],y=np.asarray(wiener_dec["RECO"]["PE"])[PE_filter]),row=1,col=2)

fig.add_trace(go.Histogram(name="GAUSS CHARGE ERROR %",x=PE_error_gauss,  xbins=dict(start=-1,end=1,size=0.01)),row=1,col=3)
fig.add_trace(go.Histogram(name="WIENER CHARGE ERROR %",x=PE_error_wiener,xbins=dict(start=-1,end=1,size=0.01)),row=1,col=3)

fig.update_layout(title_text='Charge recovery',
                xaxis_title  = "Amp. in [a.u.]",
                yaxis_title  = "Counts",
                xaxis2_title  = "Amp. in [a.u.]",
                yaxis2_title  = "Charge in [PE]",
                xaxis3_title = "Relative charge error [%PE]",
                yaxis3_title = "Counts",
                barmode='overlay')

fig.update_traces(opacity=0.75)
fig.show()

In [9]:
fig = make_subplots(rows=1, cols=1,
# subplot_titles=('Amp study', 'Charge study')
)

fig.add_trace(go.Histogram(name="GAUSS T0 ERROR %" ,x=np.asarray(raw["RECO"]["T0"])[PE_filter]-np.asarray(gauss_dec["RECO"]["T0"])[PE_filter] ,xbins=dict(start=-1,end=1,size=0.001)),row=1,col=1)
fig.add_trace(go.Histogram(name="WIENER T0 ERROR %",x=np.asarray(raw["RECO"]["T0"])[PE_filter]-np.asarray(wiener_dec["RECO"]["T0"])[PE_filter],xbins=dict(start=-1,end=1,size=0.001)),row=1,col=1)
# fig.add_trace(go.Histogram(name="WIENER T0 ERROR %",x=PE_error_wiener,xbins=dict(start=-1,end=1,size=0.01)),row=1,col=1)

fig.update_layout(title_text='T0 recovery',
                xaxis_title  = "Time in [&mu;s]",
                yaxis_title  = "Counts"
)

fig.update_traces(opacity=0.75)
fig.show()